In [1]:
# Imports
import pandas as pd
import sys
import glob
import errno
import csv
import numpy as np
from nltk.corpus import stopwords
import re
import nltk.data
import nltk
import os
from collections import OrderedDict
from subprocess import check_call
from shutil import copyfile
from sklearn.metrics import log_loss
%matplotlib inline
import matplotlib.pyplot as plt
# import mpld3
# mpld3.enable_notebook()
import seaborn as sns
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from tqdm import tqdm
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalAveragePooling1D,Lambda,Input,GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D,TimeDistributed
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from keras.layers.merge import concatenate
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.sequence import pad_sequences
from keras import initializers
from keras import backend as K
from sklearn.linear_model import SGDClassifier as sgd
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

In [2]:
pd.set_option('display.max_columns', 200)

X_train = pd.read_csv('open/train.csv', header=0,delimiter="," )
X_test = pd.read_csv( 'open/test_x.csv', header=0,delimiter="," )

authors = [0,1,2,3,4]
Y_train = LabelEncoder().fit_transform(X_train['author'])

In [3]:
# Clean data
def clean(X_train,X_test):
    X_train['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_train['text']]
    X_test['words'] = [re.sub("[^a-zA-Z]"," ", data).lower().split() for data in X_test['text']]
    return X_train,X_test
X_train,X_test = clean(X_train,X_test)

In [4]:
# Punctuation
punctuations = [{"id":1,"p":"[;:]"},
                {"id":2,"p":"[,.]"},
                {"id":3,"p":"[?]"},
                {"id":4,"p":"[!]"},
                {"id":5,"p":"[\']"},
                {"id":6,"p":"[\"]"},
                {"id":7,"p":"[;:,.?!\'\"]"}]
for p in punctuations:
    punctuation = p["p"]
    _train =  [ sentence.split() for sentence in X_train['text'] ]
    X_train['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _train]    

    _test =  [ sentence.split() for sentence in X_test['text'] ]
    X_test['punc_'+str(p["id"])] = [len([word for word in sentence if bool(re.search(punctuation, word))])*100.0/len(sentence) for sentence in _test]  

In [6]:
# tfidf - words - nb
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf,full_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_tfidf_MNB(X_train,X_test,Y_train):
    train_tfidf,test_tfidf,full_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 5])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_tfidf_MNB(X_train,X_test,Y_train)
X_train["tfidf_words_nb_0"] = pred_train[:,0]
X_train["tfidf_words_nb_1"] = pred_train[:,1]
X_train["tfidf_words_nb_2"] = pred_train[:,2]
X_train["tfidf_words_nb_3"] = pred_train[:,3]
X_train["tfidf_words_nb_4"] = pred_train[:,4]
X_test["tfidf_words_nb_0"] = pred_test[:,0]
X_test["tfidf_words_nb_1"] = pred_test[:,1]
X_test["tfidf_words_nb_2"] = pred_test[:,2]
X_test["tfidf_words_nb_3"] = pred_test[:,3]
X_test["tfidf_words_nb_4"] = pred_test[:,4]

Mean cv score :  1.206228258630992


In [7]:
# tfidf - chars - nb
def tfidfWords(X_train,X_test):
    tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,5),analyzer='char')
    full_tfidf = tfidf_vec.fit_transform(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_tfidf = tfidf_vec.transform(X_train['text'].values.tolist())
    test_tfidf = tfidf_vec.transform(X_test['text'].values.tolist())
    return train_tfidf,test_tfidf
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    train_tfidf,test_tfidf = tfidfWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 5])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test
pred_train,pred_test = do(X_train,X_test,Y_train)
X_train["tfidf_chars_nb_0"] = pred_train[:,0]
X_train["tfidf_chars_nb_1"] = pred_train[:,1]
X_train["tfidf_chars_nb_2"] = pred_train[:,2]
X_train["tfidf_chars_nb_3"] = pred_train[:,3]
X_train["tfidf_chars_nb_4"] = pred_train[:,4]
X_test["tfidf_chars_nb_0"] = pred_test[:,0]
X_test["tfidf_chars_nb_1"] = pred_test[:,1]
X_test["tfidf_chars_nb_2"] = pred_test[:,2]
X_test["tfidf_chars_nb_3"] = pred_test[:,3]
X_test["tfidf_chars_nb_4"] = pred_test[:,4]

Mean cv score :  1.8069104479701459


In [8]:
def countWords(X_train,X_test):
    count_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_MNB(X_train,X_test,Y_train):
    train_count,test_count=countWords(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 5])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_MNB(X_train,X_test,Y_train)
X_train["count_words_nb_0"] = pred_train[:,0]
X_train["count_words_nb_1"] = pred_train[:,1]
X_train["count_words_nb_2"] = pred_train[:,2]
X_train["count_words_nb_3"] = pred_train[:,3]
X_train["count_words_nb_4"] = pred_train[:,4]
X_test["count_words_nb_0"] = pred_test[:,0]
X_test["count_words_nb_1"] = pred_test[:,1]
X_test["count_words_nb_2"] = pred_test[:,2]
X_test["count_words_nb_3"] = pred_test[:,3]
X_test["count_words_nb_4"] = pred_test[:,4]

Mean cv score :  1.1789477530668817


In [9]:
# count - chars - nb
def countChars(X_train,X_test):
    count_vec = CountVectorizer(ngram_range=(1,7),analyzer='char')
    count_vec.fit(X_train['text'].values.tolist() + X_test['text'].values.tolist())
    train_count = count_vec.transform(X_train['text'].values.tolist())
    test_count = count_vec.transform(X_test['text'].values.tolist())
    return train_count,test_count
    
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

def do_count_chars_MNB(X_train,X_test,Y_train):
    train_count,test_count=countChars(X_train,X_test)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([X_train.shape[0], 5])
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = train_count[dev_index], train_count[val_index]
        dev_y, val_y = Y_train[dev_index], Y_train[val_index]
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_count)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("Mean cv score : ", np.mean(cv_scores))
    pred_full_test = pred_full_test / 5.
    return pred_train,pred_full_test

pred_train,pred_test = do_count_chars_MNB(X_train,X_test,Y_train)
X_train["count_chars_nb_0"] = pred_train[:,0]
X_train["count_chars_nb_1"] = pred_train[:,1]
X_train["count_chars_nb_2"] = pred_train[:,2]
X_train["count_chars_nb_3"] = pred_train[:,3]
X_train["count_chars_nb_4"] = pred_train[:,4]

X_test["count_chars_nb_0"] = pred_test[:,0]
X_test["count_chars_nb_1"] = pred_test[:,1]
X_test["count_chars_nb_2"] = pred_test[:,2]
X_test["count_chars_nb_3"] = pred_test[:,3]
X_test["count_chars_nb_4"] = pred_test[:,4]

Mean cv score :  5.806984089402464


In [10]:
wv = 'open/glove.6B.100d.txt'
def loadWordVecs():
    embeddings_index = {}
    f = open(wv, encoding = 'utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

def sent2vec(embeddings_index,s): # this function creates a normalized vector for the whole sentence
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stopwords.words('english')]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

def doGlove(x_train,x_test):
    embeddings_index = loadWordVecs()
    # create sentence vectors using the above function for training and validation set
    xtrain_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_train)]
    xtest_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_test)]
    xtrain_glove = np.array(xtrain_glove)
    xtest_glove = np.array(xtest_glove)
    return xtrain_glove,xtest_glove,embeddings_index

glove_vecs_train,glove_vecs_test,embeddings_index = doGlove(X_train['text'],X_test['text'])
X_train[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_train.tolist())
X_test[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_test.tolist())

  0%|                                                                                | 3/54879 [00:00<30:33, 29.94it/s]

Found 400000 word vectors.


100%|████████████████████████████████████████████████████████████████████████████| 19617/19617 [18:16<00:00, 17.89it/s]


In [11]:
earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

# NN
def doAddNN(X_train,X_test,pred_train,pred_test):
    X_train["nn_0"] = pred_train[:,0]
    X_train["nn_1"] = pred_train[:,1]
    X_train["nn_2"] = pred_train[:,2]
    X_train["nn_3"] = pred_train[:,3]
    X_train["nn_4"] = pred_train[:,4]
    
    X_test["nn_0"] = pred_test[:,0]
    X_test["nn_1"] = pred_test[:,1]
    X_test["nn_2"] = pred_test[:,2]
    X_test["nn_3"] = pred_test[:,3]
    X_test["nn_4"] = pred_test[:,4]
    return X_train,X_test

def initNN(nb_words_cnt,max_len):
    model = Sequential()
    model.add(Embedding(nb_words_cnt,32,input_length=max_len))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,
                     5,
                     padding='valid',
                     activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def doNN(X_train,X_test,Y_train):
    max_len = 70
    nb_words = 10000
    
    print('Processing text dataset')
    texts_1 = []
    for text in X_train['text']:
        texts_1.append(text)

    print('Found %s texts.' % len(texts_1))
    test_texts_1 = []
    for text in X_test['text']:
        test_texts_1.append(text)
    print('Found %s texts.' % len(test_texts_1))
    
    tokenizer = Tokenizer(num_words=nb_words)
    tokenizer.fit_on_texts(texts_1 + test_texts_1)
    sequences_1 = tokenizer.texts_to_sequences(texts_1)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)

    xtrain_pad = pad_sequences(sequences_1, maxlen=max_len)
    xtest_pad = pad_sequences(test_sequences_1, maxlen=max_len)
    del test_sequences_1
    del sequences_1
    nb_words_cnt = min(nb_words, len(word_index)) + 1

    # we need to binarize the labels for the neural net
    ytrain_enc = np_utils.to_categorical(Y_train)
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], 5])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN(nb_words_cnt,max_len)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=4, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_pad)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN(X_train,X_test,pred_train,pred_full_test/5)

## NN Glove

def doAddNN_glove(X_train,X_test,pred_train,pred_test):
    X_train["nn_glove_0"] = pred_train[:,0]
    X_train["nn_glove_1"] = pred_train[:,1]
    X_train["nn_glove_2"] = pred_train[:,2]
    X_train["nn_glove_3"] = pred_train[:,3]
    X_train["nn_glove_4"] = pred_train[:,4]
    X_test["nn_glove_0"] = pred_test[:,0]
    X_test["nn_glove_1"] = pred_test[:,1]
    X_test["nn_glove_2"] = pred_test[:,2]
    X_test["nn_glove_3"] = pred_test[:,3]
    X_test["nn_glove_4"] = pred_test[:,4]
    return X_train,X_test

def initNN_glove():
    # create a simple 3 layer sequential neural net
    model = Sequential()

    model.add(Dense(128, input_dim=100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(5))
    model.add(Activation('softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def doNN_glove(X_train,X_test,Y_train,xtrain_glove,xtest_glove):
    # scale the data before any neural net:
    scl = preprocessing.StandardScaler()
    ytrain_enc = np_utils.to_categorical(Y_train)
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    xtrain_glove = scl.fit_transform(xtrain_glove)
    xtest_glove = scl.fit_transform(xtest_glove)
    pred_train = np.zeros([xtrain_glove.shape[0], 5])
    
    for dev_index, val_index in kf.split(xtrain_glove):
        dev_X, val_X = xtrain_glove[dev_index], xtrain_glove[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN_glove()
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=10, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_glove)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN_glove(X_train,X_test,pred_train,pred_full_test/5)

# Fast Text

def doAddFastText(X_train,X_test,pred_train,pred_test):
    X_train["ff_0"] = pred_train[:,0]
    X_train["ff_1"] = pred_train[:,1]
    X_train["ff_2"] = pred_train[:,2]
    X_train["ff_3"] = pred_train[:,3]
    X_train["ff_4"] = pred_train[:,4]
    X_test["ff_0"] = pred_test[:,0]
    X_test["ff_1"] = pred_test[:,1]
    X_test["ff_2"] = pred_test[:,2]
    X_test["ff_3"] = pred_test[:,3]
    X_test["ff_4"] = pred_test[:,4]
    return X_train,X_test


def initFastText(embedding_dims,input_dim):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def preprocessFastText(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocessFastText(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

def doFastText(X_train,X_test,Y_train):
    min_count = 2

    docs = create_docs(X_train)
    tokenizer = Tokenizer(lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

    tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    docs = tokenizer.texts_to_sequences(docs)

    maxlen = 300

    docs = pad_sequences(sequences=docs, maxlen=maxlen)
    input_dim = np.max(docs) + 1
    embedding_dims = 20

    # we need to binarize the labels for the neural net
    ytrain_enc = np_utils.to_categorical(Y_train)

    docs_test = create_docs(X_test)
    docs_test = tokenizer.texts_to_sequences(docs_test)
    docs_test = pad_sequences(sequences=docs_test, maxlen=maxlen)
    xtrain_pad = docs
    xtest_pad = docs_test
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], 5])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initFastText(embedding_dims,input_dim)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=25, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(docs_test)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddFastText(X_train,X_test,pred_train,pred_full_test/5)

X_train,X_test = doFastText(X_train,X_test,Y_train)
X_train,X_test = doNN(X_train,X_test,Y_train)
X_train,X_test = doNN_glove(X_train,X_test,Y_train,glove_vecs_train,glove_vecs_test)

Epoch 1/25
1372/1372 [==============================] - 107s 78ms/step - loss: 1.5135 - accuracy: 0.3480 - val_loss: 1.4208 - val_accuracy: 0.4515
Epoch 2/25
1372/1372 [==============================] - 106s 77ms/step - loss: 1.2441 - accuracy: 0.5824 - val_loss: 1.1198 - val_accuracy: 0.6204
Epoch 3/25
1372/1372 [==============================] - 107s 78ms/step - loss: 0.9624 - accuracy: 0.6914 - val_loss: 0.9167 - val_accuracy: 0.6983
Epoch 4/25
1372/1372 [==============================] - 107s 78ms/step - loss: 0.7770 - accuracy: 0.7570 - val_loss: 0.7917 - val_accuracy: 0.7394
Epoch 5/25
1372/1372 [==============================] - 107s 78ms/step - loss: 0.6479 - accuracy: 0.8017 - val_loss: 0.7058 - val_accuracy: 0.7711
Epoch 6/25
1372/1372 [==============================] - 107s 78ms/step - loss: 0.5487 - accuracy: 0.8361 - val_loss: 0.6449 - val_accuracy: 0.7849
Epoch 7/25
1372/1372 [==============================] - 107s 78ms/step - loss: 0.4693 - accuracy: 0.8607 - val_loss: 0

1372/1372 [==============================] - 118s 86ms/step - loss: 0.6635 - accuracy: 0.7959 - val_loss: 0.7207 - val_accuracy: 0.7665
Epoch 6/25
1372/1372 [==============================] - 116s 85ms/step - loss: 0.5626 - accuracy: 0.8317 - val_loss: 0.6575 - val_accuracy: 0.7829
Epoch 7/25
1372/1372 [==============================] - 116s 85ms/step - loss: 0.4819 - accuracy: 0.8596 - val_loss: 0.6082 - val_accuracy: 0.7984
Epoch 8/25
1372/1372 [==============================] - 118s 86ms/step - loss: 0.4165 - accuracy: 0.8817 - val_loss: 0.5705 - val_accuracy: 0.8110
Epoch 9/25
1372/1372 [==============================] - 119s 87ms/step - loss: 0.3616 - accuracy: 0.8982 - val_loss: 0.5435 - val_accuracy: 0.8202
Epoch 10/25
1372/1372 [==============================] - 117s 85ms/step - loss: 0.3158 - accuracy: 0.9128 - val_loss: 0.5175 - val_accuracy: 0.8258
Epoch 11/25
1372/1372 [==============================] - 117s 85ms/step - loss: 0.2767 - accuracy: 0.9264 - val_loss: 0.5017 - v

1372/1372 [==============================] - 3s 2ms/step - loss: 1.2302 - val_loss: 1.2054
Epoch 8/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2210 - val_loss: 1.2060
Epoch 1/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.4883 - val_loss: 1.2769
Epoch 2/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.3150 - val_loss: 1.2444
Epoch 3/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2811 - val_loss: 1.2270
Epoch 4/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2663 - val_loss: 1.2146
Epoch 5/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2510 - val_loss: 1.2085
Epoch 6/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2397 - val_loss: 1.1993
Epoch 7/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2287 - val_loss: 1.1956
Epoch 8/10
1372/1372 [==============================] - 3s 2ms/step - loss: 1.2235 - val_loss

In [12]:
# Final Model
# XGBoost
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 5
    param['silent'] = 1
    param['num_class'] = 5
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

def do(X_train,X_test,Y_train):
    drop_columns=["text","words"]
    x_train = X_train.drop(drop_columns+['author'],axis=1)
    x_test = X_test.drop(drop_columns,axis=1)
    y_train = Y_train
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([x_train.shape[0], 5])
    for dev_index, val_index in kf.split(x_train):
        dev_X, val_X = x_train.loc[dev_index], x_train.loc[val_index]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, x_test, seed_val=0, colsample=0.7)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv scores : ", cv_scores)
    print("Mean cv score : ", np.mean(cv_scores))
    return pred_full_test/5
result = do(X_train,X_test,Y_train)

[00:24:06] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:1.44020	test-mlogloss:1.44360
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
[20]	train-mlogloss:0.48894	test-mlogloss:0.51624
[40]	train-mlogloss:0.36624	test-mlogloss:0.41435
[60]	train-mlogloss:0.32861	test-mlogloss:0.39657
[80]	train-mlogloss:0.30391	test-mlogloss:0.39082
[100]	train-mlogloss:0.28196	test-mlogloss:0.38816
[120]	train-mlogloss:0.26241	test-mlogloss:0.38666
[140]	train-mlogloss:0.24459	test-mlogloss:0.38565
[160]	train-mlogloss:0.22787	test-mlogloss:0.38532
[180]	train-

In [23]:
# cv scores :  [0.37971865148208395, 0.3913440671935568, 0.39145263387349694, 0.38940780721717677, 0.38413344802020066]
# cv scores :  [0.38404312992420286, 0.3863583009575495, 0.39057757107939883, 0.3823430674453724, 0.38250879232159984]
a =  [0.37971865148208395, 0.3913440671935568, 0.39145263387349694, 0.38940780721717677, 0.38413344802020066]
b = [0.38404312992420286, 0.3863583009575495, 0.39057757107939883, 0.3823430674453724, 0.38250879232159984]
print(sum(a)/5)
print(sum(b)/5)

0.38721132155730303
0.38516617234562467


In [14]:
sample_submission=pd.read_csv('open/sample_submission.csv', encoding='utf-8')
sample_submission[['0', '1', '2', '3', '4']] = result
sample_submission.to_csv("kg_3_1122.csv", index=False)
sample_submission

,index,0,1,2,3,4
0,0,0.001581,0.949521,0.032718,0.007405,0.008775
1,1,0.003760,0.986013,0.001274,0.001857,0.007095
2,2,0.998662,0.000541,0.000154,0.000142,0.000501
3,3,0.002060,0.032035,0.963130,0.000627,0.002147
4,4,0.983971,0.004545,0.002014,0.004365,0.005105
...,...,...,...,...,...,...
19612,19612,0.002113,0.996147,0.000868,0.000574,0.000297
19613,19613,0.001742,0.000617,0.000579,0.000191,0.996871
19614,19614,0.000516,0.998709,0.000332,0.000246,0.000198
19615,19615,0.000287,0.998773,0.000489,0.000334,0.000117
